In [ ]:
import numpy as np
import uproot, json

from matrix import loadMatrices, baseTransform

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

In [ ]:
with open("comparisonData/modules-1.00/rawData/sensorToModule.json", "r") as f:
    sensorIDdict=json.load(f)
    sectorIDlut = np.empty(len(sensorIDdict))

    # create a look up table for fast sensorID -> sectorID translation
    for key, value in sensorIDdict.items():
        sectorIDlut[int(key)] = value

def getSectorFromSensorID(sensorID):
    return sensorIDdict[str(sensorID)]

In [ ]:
def readRecoHitsFromRootFiles(filename, maxNoOfFiles=0):

    # make empty 3D (n x 4 x 4) result array.
    # n tracks, 4 reco hits per track (all others are discarded),
    # 4 vales per reco (sector id, x, y, z)
    resultTracks = np.empty((0, 4, 4))

    runIndex = 0
    for arrayDict in uproot.iterate(
        filename,
        [
            "LMDHitsMerged.fSensorID",
            "LMDHitsMerged.fX",
            "LMDHitsMerged.fY",
            "LMDHitsMerged.fZ",
        ],
        library="np",
        allow_missing=True,  # some files may be empty, skip those
    ):
        ids = arrayDict["LMDHitsMerged.fSensorID"]
        recoX = arrayDict["LMDHitsMerged.fX"]
        recoY = arrayDict["LMDHitsMerged.fY"]
        recoZ = arrayDict["LMDHitsMerged.fZ"]

        # create mask for events that have exactly 4 hits
        mask = [event.size == 4 for event in ids]

        # make a real 2D array from array[array[ ]]
        maskedIDs = np.stack(ids[mask])

        # calculate module number from sensor IDs
        # use look up table for that
        # thank you TheodrosZelleke for this insanely smart idea
        # https://stackoverflow.com/a/14448935
        sectorIDfromLut = sectorIDlut[maskedIDs]

        # make arrays of arrays to 2d arrays
        xFlat = np.stack(recoX[mask])
        yFlat = np.stack(recoY[mask])
        zFlat = np.stack(recoZ[mask])

        # transpose to assemble and transpose again
        theseTracks = np.array([sectorIDfromLut.T, xFlat.T, yFlat.T, zFlat.T]).T

        # sort every 4-hit combo by z value. do this for every event.
        # this is REQUIRED for the aligner
        # I know it looks weird seeing a list comprehension here instead of something
        # more NumPythonic, but this seems to be fastest after all
        # the numpy version would probably look like this:
        # sortedResultArray = theseTracks[:,theseTracks[:,:,3].argsort()]
        # or 
        # sortedResultArray = np.einsum('iijk->ijk', theseTracks[:,theseTracks[:,:,0].argsort()])
        # and be 20x slower (and require MUCH more memory)
        sortedResultArray = np.array([subarray[subarray[:,3].argsort()] for subarray in theseTracks])

        # stack this files' content with the others
        resultTracks = np.vstack((resultTracks, sortedResultArray))

        runIndex += 1
        if runIndex == maxNoOfFiles:
            break

    return resultTracks


In [ ]:
class  CorridorFitter:
    def __init__(self, tracks):
        self.tracks = tracks
        self.nTrks = len(self.tracks)
        self.useAnchor = False

    def useAnchorPoint(self, point):
        assert len(point) == 3 or len(point) == 4
        self.anchorPoint = point
        self.useAnchor = True

    def fitTracksSVD(self):

        self.fittedTrackArr = np.zeros((self.nTrks, 2, 3))

        for i in range(self.nTrks):
            # cut fourth entry, sometimes this is the sensorID or homogeneous coordinate
            trackRecos = self.tracks[i][:, :3]

            if self.useAnchor:
                trackRecos = np.vstack((self.anchorPoint[:3], trackRecos))

            # see https://stackoverflow.com/questions/2298390/fitting-a-line-in-3d
            meanPoint = trackRecos.mean(axis=0)
            _, _, vv = np.linalg.svd(trackRecos - meanPoint)

            self.fittedTrackArr[i][0] = meanPoint

            # flip tracks that are fitted backwards
            if vv[0, 2] < 0:
                self.fittedTrackArr[i][1] = -vv[0]
            else:
                self.fittedTrackArr[i][1] = vv[0]

        return self.fittedTrackArr

In [ ]:
# ? cuts on track x,y direction
def dynamicTrackCut(newTracks, cutPercent=1):
    com = np.average(newTracks[:, 1, :3], axis=0)

    # shift newhit2 by com of differences
    newhit2 = newTracks[:, 1, :3] - com
    newDist = np.power(newhit2[:, 0], 2) + np.power(newhit2[:, 1], 2)

    cut = int(len(newhit2) * (cutPercent / 100.0))
    newTracks = newTracks[newDist.argsort()]
    newTracks = newTracks[:-cut]
    return newTracks

# ? cuts on reco-track distance
def dynamicRecoTrackDistanceCut(newTracks, cutPercent=1):

    # don't worry, numpy arrays are copied by reference
    tempTracks = newTracks

    for i in range(4):
        trackPosArr = tempTracks[:, 0, :3]
        trackDirArr = tempTracks[:, 1, :3]
        recoPosArr = tempTracks[:, 2 + i, :3]

        # norm momentum vectors, this is important for the distance formula!
        trackDirArr = (
            trackDirArr / np.linalg.norm(trackDirArr, axis=1)[np.newaxis].T
        )

        # vectorized distance calculation
        tempV1 = trackPosArr - recoPosArr
        tempV2 = (tempV1 * trackDirArr).sum(axis=1)
        dVec = tempV1 - tempV2[np.newaxis].T * trackDirArr
        dVec = dVec[:, :2]
        newDist = np.power(dVec[:, 0], 2) + np.power(dVec[:, 1], 2)

        # cut
        cut = int(len(dVec) * (cutPercent / 100.0))
        tempTracks = tempTracks[newDist.argsort()]
        tempTracks = tempTracks[:-cut]

    return tempTracks

In [ ]:
import icp


def getMatrix(trackPositions, recoPositions, use2D=False):

    # use 2D, use only in LMD local!
    if use2D:
        T, _, _ = icp.best_fit_transform(
            trackPositions[..., :2], recoPositions[..., :2]
        )

        # homogenize
        resultMat = np.identity(4)
        resultMat[:2, :2] = T[:2, :2]
        resultMat[:2, 3] = T[:2, 2]
        return resultMat

    else:
        T, _, _ = icp.best_fit_transform(trackPositions, recoPositions)
        return T


In [ ]:
def alignSectorICP(sectorRecos, sector, maxNoTrks=0):
    """
    @param sectorRecos np array for track positon, direction and recos
    """

    iterations = 5

    # TODO: add to config!
    preTransform = False

    detectorMatrices = loadMatrices("comparisonData/oldInput/detectorMatricesIdeal.json")
    avgMisalignments = loadMatrices(
        "comparisonData/oldInput/moduleAlignment/avgMisalign-1.00.json"
    )

    with open('comparisonData/oldInput/moduleAlignment/anchorPoints.json') as f:
        anchorPoints = json.load(f)

    with open('comparisonData/oldInput/moduleAlignment/sectorPaths.json') as f:
        allModulePaths = json.load(f)

    # get relevant module paths
    modulePaths = allModulePaths[str(sector)]

    # make 4x4 matrices to module positions
    moduleMatrices = np.zeros((4, 4, 4))
    for i in range(len(modulePaths)):
        path = modulePaths[i]
        moduleMatrices[i] = np.array(detectorMatrices[path])

    # * use average misalignment
    averageShift = avgMisalignments[str(sector)]

    # assing given tracks to internal variable
    newTracks = sectorRecos

    # use only N tracks:
    if maxNoTrks > 0:
        newTracks = newTracks[:maxNoTrks]

    sectorString = str(sector)
    # transform all recos to LMD local
    if preTransform:
        matToLMD = np.linalg.inv(np.array(detectorMatrices["/cave_1/lmd_root_0"]))
        for i in range(4):
            newTracks[:, i + 2] = (matToLMD @ newTracks[:, i + 2].T).T

    # transform anchorPoints to PANDA global
    else:
        matFromLMD = np.array(detectorMatrices["/cave_1/lmd_root_0"])
        anchorPoints[sectorString] = matFromLMD @ anchorPoints[sectorString]

    print(f"==================================================")
    print(f"        module aligner for sector {sector}")
    print(f"==================================================")

    print(f"number of tracks: {len(newTracks)}")
    print(f"anchor point: {anchorPoints[sectorString]}")

    # do a first track fit, otherwise we have no starting tracks
    recos = newTracks[:, 2:6]
    corrFitter = CorridorFitter(recos)
    corrFitter.useAnchorPoint(anchorPoints[sectorString][:3])
    resultTracks = corrFitter.fitTracksSVD()

    # update current tracks
    newTracks[:, 0, :3] = resultTracks[:, 0]
    newTracks[:, 1, :3] = resultTracks[:, 1]

    # prepare total matrices
    totalMatrices = np.zeros((4, 4, 4))
    for i in range(4):
        totalMatrices[i] = np.identity(4)

    # * =========== iterate cuts and calculation
    for iIteration in range(iterations):
        print(f"running iteration {iIteration}, {len(newTracks)} tracks remaining...")

        newTracks = dynamicRecoTrackDistanceCut(newTracks)

        # 4 planes per sector
        for i in range(4):
            trackPosArr = newTracks[:, 0, :3]
            trackDirArr = newTracks[:, 1, :3]
            recoPosArr = newTracks[:, 2 + i, :3]

            # norm momentum vectors, this is important for the distance formula!
            trackDirArr = (
                trackDirArr / np.linalg.norm(trackDirArr, axis=1)[np.newaxis].T
            )

            # vectorized distance calculation
            tempV1 = trackPosArr - recoPosArr
            tempV2 = (tempV1 * trackDirArr).sum(axis=1)
            dVec = tempV1 - tempV2[np.newaxis].T * trackDirArr

            # the vector thisReco+dVec now points from the reco hit to the intersection of the track and the sensor
            pIntersection = recoPosArr + dVec

            # we want FROM tracks TO recos
            T0inv = getMatrix(recoPosArr, pIntersection, preTransform)
            totalMatrices[i] = T0inv @ totalMatrices[i]

            # transform recos, MAKE SURE THEY ARE SORTED
            newTracks[:, i + 2] = (T0inv @ newTracks[:, i + 2].T).T

        # direction cut again
        if iIteration < 3:
            newTracks = dynamicTrackCut(newTracks, 1)

        # do track fit
        corrFitter = CorridorFitter(newTracks[:, 2:6])
        resultTracks = corrFitter.fitTracksSVD()

        # update current tracks
        newTracks[:, 0, :3] = resultTracks[:, 0]
        newTracks[:, 1, :3] = resultTracks[:, 1]

    # * =========== store matrices
    # 4 planes per sector

    result = {}

    # transform the alignment matrices back INTO the system of their respective module
    # since that's where FAIRROOT applies them
    for i in range(4):
        # ideal module matrices!
        toModMat = moduleMatrices[i]
        if preTransform:
            totalMatrices[i] = baseTransform(totalMatrices[i], np.linalg.inv(matToLMD))
            totalMatrices[i] = baseTransform(totalMatrices[i], np.linalg.inv(toModMat))
        else:
            totalMatrices[i] = baseTransform(totalMatrices[i], np.linalg.inv(toModMat))

        # add average shift (external measurement)
        totalMatrices[i] = totalMatrices[i] @ averageShift
        result[modulePaths[i]] = totalMatrices[i]

    
    print(f"-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    print(f"        module aligner for sector {sector} done!         ")
    print(f"-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    return result


In [ ]:
filename = "comparisonData/modules-1.00/rawData/Lumi_recoMerged_*.root"
resultArray = readRecoHitsFromRootFiles(filename, 6)

alignmentMatices = {}

# translate the reco hits to format for module Aligner
for i in range(10):
    # apply mask so only one sector is chosen
    # careful, this only checks if the first hit
    # is in the correct sector!
    sectorMask = resultArray[:, 0, 0] == i

    # create new empty array thay has the required structure
    sectorTracksXYZ = resultArray[sectorMask][:, :, 1:4]
    nTrks = len(sectorTracksXYZ)

    # assign recos from input array to new array for this sector
    trackVectorForAligner = np.ones((nTrks, 6, 4))
    trackVectorForAligner[:, 2:6, :3] = sectorTracksXYZ[:, 0:4]
    alignmentMatices |= alignSectorICP(trackVectorForAligner, i)


In [ ]:
path = '/cave_1/lmd_root_0/half_0/plane_1/module_3'

print('This time:')
print(alignmentMatices[path]*1e4)

print('--------------------------------------------------------\nLast time:')
mOld = loadMatrices('comparisonData/modules-1.00/alMat-moduleAlignment-1.00.json')
print(mOld[path]*1e4)

print('--------------------------------------------------------\nActual:')
mActual = loadMatrices('comparisonData/misMatrices/misMat-modules-1.00.json')
print(mActual[path]*1e4)
